## Chapter 6 - Decision Trees
---

* **1. What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with 1 million instances?**
> Since decision trees are generaly fairly balanced, the depth can be taken as the number of nodes you must pass through to traverse the tree from root to a leaf. This requires going through $O({\log_2 m})\approx 20$ nodes, where $m = 10^6$.

* **2. Is a node’s Gini impurity generally lower or greater than its parent’s? Is it generally lower/greater, or always lower/greater?**
> Generally, a node will have a lower Gini impurity than it's parents' as the point of splitting a node is to further partition the instances such that they're grouped into purer nodes which will likely be better predictors; the CART algorithm's cost function ensures the weighted sum of the children's Gini impurities is minimised. Theoretically a node's impurity could definitely be higher than it's parents' but in practice, this doesn't usually happen as if scikit-learn can't find a split that reduces impurity it stops training. *(Note - Exercise solutions note one exception to this, which is if one child is smaller than the other it can have a higher Gini impurity than its parent provided this increase is more than compensated for by a decrease in the other child's impurity.)*

* **3. If a Decision Tree is overfitting the training set, is it a good idea to try decreasing max_depth?**
> Yes, this would be a good idea. A large `max_depth` results in a very complex model with many decision boundaries which may result in poor generalisation to unseen data. Decreasing this hyperparameter will reduce the number of decision boundaries created, regularising the model and making it more able to generalise.

* **4. If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?**
> No, as doing so will likely have no effect. One of the major positives about working with decision trees is that they can be used with unscaled, uncentred data, unlike many other ML models which require both of these preprocessing steps. If the tree is underfitting, the best course of action is to increase the `max_depth` hyperparameter.

* **5. If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?**
> The training algorithm complexity is given by $O(n \times m{\log_2 m})$. Due to the number of instances increasing from $10^6$ to $10^7$ but the number of features $n$ staying the same, the training time is larger by a factor $k$ which can be defined by $k = \frac{n \times 10^7{\log_2 10^7}}{n \times 10^6{\log_2 10^6}}$, which simplifies to $k = 10 \times \frac{7{\log_2 10}}{6{\log_2 10}}$, which evaluates to $11 \frac{2}{3}$, hence the tree will take 11 hours and 40 minutes to train.

* **6. If your training set contains 100,000 instances, will setting presort=True speed up training?**
> No. Typically, `presort=True` will speed up computation for training sets of a few thousand instances but for training sets larger than that it actually slows the process down considerably.

* **7. Train and fine-tune a Decision Tree for the moons dataset.**
>* **a. Generate a moons dataset using make_moons(n_samples=10000, noise=0.4).**
>* **b. Split it into a training set and a test set using train_test_split().**
>* **c. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.**
>* **d. Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.**

After loading in the data, splitting it into training and test sets and importing the functions we'll need, we can get to work setting up the hyperparameter options to feed into the Grid Search.

In [82]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ShuffleSplit
from scipy.stats import mode

np.random.seed(12)

X, y = make_moons(n_samples=10000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

dt_clf = DecisionTreeClassifier()
param_grid = {
        'max_depth':[5, 10, 15],
        'max_leaf_nodes':[15, 25, 35]    
    }

dt_grid = GridSearchCV(dt_clf, param_grid, cv=3, scoring='accuracy')
dt_grid.fit(X_train, y_train)
print('Optimal parameters are:', dt_grid.best_params_)
print('Accuracy on training set:', dt_grid.score(X_train, y_train))
y_pred = dt_grid.predict(X_test)
print('Accuracy on test set:', accuracy_score(y_test, y_pred))

Optimal parameters are: {'max_depth': 5, 'max_leaf_nodes': 15}
Accuracy on training set: 0.86025
Accuracy on test set: 0.857


Here we see that after a small amount of hyperparameter tuning we've ended up with a model which actually performs relatively well, giving an accuracy of 85.7% on the test set.

* **8. Grow a forest.**
>* **a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn’s `ShuffleSplit` class for this.**
>* **b. Train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.**
>* **c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s `mode()` function for this). This gives you majority-vote predictions over the test set.**
>* **d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!**

Firstly, we use `ShuffleSplit`, specifying 1,000 splits of 100 random instances and add these subsets to a master list. As expected, when we train each of these subsets and average their accuracies, we end up with a reduced accuracy score of 78.9% due to the reduced amount of training data.

In [83]:
X_train_subsets, X_test_subsets, trees, scores = [], [], [], []

ss = ShuffleSplit(n_splits=1000, train_size=100, random_state=42)

for train_index, test_index in ss.split(X_train):
    X_subset, y_subset = X_train[train_index], y_train[train_index]
    X_train_subsets.append(X_subset)
    dt_clf_best = DecisionTreeClassifier(max_depth=10, max_leaf_nodes=25)
    trees.append(dt_clf_best.fit(X_subset, y_subset))
    y_pred = dt_clf_best.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))
    
print('Average accuracy on test set across the 1000 trees:',
     np.mean(scores))

C:\Users\RR5806\AppData\Local\conda\conda\envs\python-ml\lib\site-packages\sklearn\model_selection\_split.py:1788: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Average accuracy on test set across the 1000 trees: 0.7893365


Now we find the most frequent prediction among the forest of decision trees for each instance in the test set by using `scipy.stats.mode` on the array of predictions. This returns an accuracy of 86.9%, which is 1.2% higher than the result the Decision Tree gave us, so the Random Forest has worked as expected.

In [85]:
all_preds = np.zeros((1000, 2000))

for i, tree in enumerate(trees):
    tree_preds = tree.predict(X_test)
    all_preds[i, :] = tree_preds
    
y_pred = mode(all_preds, axis=0)[0][0]
print('Forest accuracy score on test set:', accuracy_score(y_test, y_pred))

Forest accuracy score on test set: 0.869
